# Practical 8

## Aim

To learn how to examine and interpret interaction between two variables in a logistic model. 

In [ ]:
library(tidyverse)

## Reading in the dataset and identifying relevant variables

In this practical session we will use a dataset from the study of helminths in Uganda.

To read in the dataset, type:

In [ ]:
library(haven)

In [ ]:
helminths_df <- read_dta("Data_files-20211113/helminths.dta")

In this session we will focus again on hookworm infection as the main exposure and severe anaemia as the outcome. If you are starting this practical, you may need to make sure it is called hookworm. If it is currently called hk_bin. type:

In [ ]:
helminths_df_2 <- helminths_df %>%
    mutate(hookworm = hk_bin)

This has now renamed the variable describing hookworm infection status to hookworm. We will examine whether the effect of hookworm on severe anaemia is modified by age. The aim of all statistical modelling exercises is to obtain the best measure of effect for the exposure of interest. There is no point in having a complex model which is difficult to interpret if a simpler one is statistically almost as good. So our strategy will be to test a model with interaction between hookworm and agegrp.  If it does not significantly improve the model, tested via the likelihood ratio test (LRT), then a simpler model with no interaction will be chosen.   
 
First a recap of the variables of interest:

**anaemic_sev**  
    coded: 0=no, 1=yes 
 
**hookworm** is the variable name for hookworm infection status  
    coded:  0=uninfected, 1=infected 

**agegrp** is the variable name for age-group 
    coded: 0=<20, 1=20-24, 2=25-29, 3=30+ 

**malaria** 
    coded: 0=uninfected, 1=infected 


To illustrate with a less complex model, use recode with age to create a binary variable. This is only less complex in that there will be fewer parameters in the model. (It is possible to test interactions with more than two categories in a variable, but we are trying here to keep things simpler at first.) To create a new variable where age takes 2  levels, type: 

In [ ]:
helminths_df_3 <- helminths_df_2 %>%
    mutate(agebin = as.factor(if_else(agegrp < 2, 0, 1)))

Check the recoding of the new variable agebin by tabulating it against the original 
variable agegrp.  Type:

In [ ]:
table(helminths_df_3$agegrp, helminths_df_3$agebin)

## Specifying a logistic model with an interaction term

In Practical 7 we produced a logistic model that assumed proportional odds. This assumed that the effect of hookworm infection on severe anaemia is the same for all levels of age. If the effect of hookworm is modified by age, i.e. there is interaction between the two variables, we should include the interaction in our model.

To produce a model with interaction in `glm` insert `*` between the two variables for which the interaction parameter(s) are required. Type:

In [ ]:
anaemia_agebin_interaction_glm <-
glm(anaemic_sev ~ agebin * hookworm,
    data = helminths_df_3,
    family = binomial)

In [ ]:
summary(anaemia_agebin_interaction_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_agebin_interaction_glm), confint(anaemia_agebin_interaction_glm)))

We will return to the interpretation of this output shortly.

## Likelihood ratio test for interaction  

Before attempting to interpret interaction terms let’s test this interaction formally using a likelihood ratio test.  Type the following commands: 

In [ ]:
anaemia_agebin_glm <-
glm(anaemic_sev ~ agebin + hookworm,
    data = helminths_df_3,
    family = binomial)

In [ ]:
library(lmtest)

In [ ]:
lrtest(anaemia_agebin_interaction_glm, anaemia_agebin_glm)

The LRT tests the null hypothesis that the model with the interaction term does not improve the model fit when compared to the model without the interaction term, or, put another way, that the two models fit equally well. There is no strong evidence (P=0.672) against this null hypothesis, therefore it would be reasonable to assume that there is no strong evidence for interaction and to accept the simpler model.  

In [ ]:
summary(anaemia_agebin_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_agebin_glm), confint(anaemia_agebin_glm)))

From this model we would conclude that the odds of severe anaemia in hookworm infected women are 2.8 times the odds of severe anaemia in hookworm uninfected women irrespective of age, and that the odds of severe anaemia in those aged 25+ years are 0.7 times the odds of severe anaemia in those aged <25 years regardless of hookworm infection status. 

## Interpreting an interaction model 

There is very little evidence against the null hypothesis of no interaction in this example and we would therefore usually choose to assume that the proportional odds model is reasonable. However, for the purpose of demonstrating how to analyse and interpret interactions we will proceed with the interaction model. 

Let’s now return to the output of the interaction model from page 59.  

In [ ]:
summary(anaemia_agebin_interaction_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_agebin_interaction_glm), confint(anaemia_agebin_interaction_glm)))

This model contains an interaction between age and hookworm infection status. This means that there is no longer an overall age effect regardless of hookworm infection status because the effect of age differs according to hookworm infection status (and vice versa, the hookworm effect depends on age group). 

The coefficients for the main effects are interpreted as the effect of the exposure when all other exposures involved in the interaction in the model are equal to their baseline categories. The two exposures involved in the interaction in this model are age group and hookworm infection. 

As an aside, note that if we had also included a third variable in this model (say, maternal education), as a main effect only, we would interpret the coefficients for the main effect of agebin as the effect of agebin when hookworm infection was equal to its baseline category, but now adjusted for maternal education. Similarly, the coefficient for the main effect of hookworm infection would be interpreted as the effect of hookworm infection, adjusted for maternal education, and when agebin is equal to its baseline category. 

The coefficient for `agebin` is 0.75 which represents the effect of age group in the baseline category for hookworm. Therefore, the odds of severe anaemia in those aged 25+ are 0.75 times the odds of severe anaemia in those aged <25 years who are uninfected with hookworm. 

Similarly the odds of severe anaemia in hookworm infected women are 2.95 times the odds of severe anaemia in hookworm uninfected women in the <25 year age-group. These are known as stratum-specific effects: they are the effects within specific strata (meaning groups or combinations of groups). Let’s divide the age and hookworm categories into four strata: 

[A] = [age <25, hookworm uninfected] 

[B] = [age 25+, hookworm uninfected] 

[C] = [age <25, hookworm infected] 

[D] = [age 25+, hookworm infected] 

Here [A] corresponds to the strata where both exposures equal their baseline categories, and the coefficient for `agebin` represents the comparison [B vs A] and the coefficient for hookworm represents the comparison [C vs A].

The 3rd row of the output table corresponds to the interaction parameter (which `R` refers to as `agebin1:hookworm`). The interaction coefficient is the additional effect, over and above the main effects, that occurs when both exposures are “present”. 
 
For example, the odds ratio for the comparison [D vs A] is obtained as: 

OR = 0.75 × 2.95 × 0.88 = 1.95.  
 
There are still some stratum-specific estimates that we have not yet evaluated. For example we know the odds ratio in hookworm infected compared to uninfected women is 2.95 in the <25 years age group but what is it in the 25+ age group? To determine this we must multiply the appropriate coefficients from the model. This particular estimate corresponds to the comparison [D vs B]. Age is the same in both of these strata therefore we do not need to include a main effect for age. We are comparing hookworm infection vs “no infection” so we do require the main effect of hookworm. Both exposures are 
“present” in strata [D] so we also need to include the interaction parameter. Therefore the odds ratio is calculated as OR = 2.95 × 0.88 = 2.60, i.e. in the 25+ age group, the odds of severe anaemia in hookworm infected women are 2.60 times the odds of severe anaemia in women without hookworm infection. 
 
Following similar logic, in hookworm infected women the odds of severe anaemia in those aged 25+ are 0.75 × 0.88 = 0.66 times the odds of severe anaemia in those aged <25 years (This corresponds to the comparison [D vs C]. 
 
Let’s just review the results from the models with and without the interaction. The simpler model (the proportional odds model, since it assumes the odds of severe anaemia are the same regardless of the categories of the other exposure) gives the overall age effect as OR = 0.70. On the other hand the interaction model gives the stratum-specific (i.e. hookworm infection-specific) age effects as OR = 0.75 for hookworm uninfected and OR = 0.66 for hookworm infected women. 
 
Notes: 

1. The formal test for interaction suggested very little evidence against the null hypothesis, however the actual stratum-specific estimates are somewhat 
different to each other. It is important to take away from this that the size of the p-value for the test for interaction does not indicate the magnitude of the interaction effect. 
2. The overall effect lies somewhere between the two stratum-specific effects; it is essentially a weighted average of the two. 
3. While the two stratum specific odds ratios appear somewhat different, they both point in the same direction (i.e. are <1).  
4. Also, we have not calculated confidence intervals. If the confidence intervals overlap, or include the other stratum specific value, we might reconsider how different we feel the two stratum specific values are.  
5. Both the model with and without interaction estimates the underlying ‘true’ effects in the population, and there is no strong evidence that one model is better than the other.  
6. Generally speaking, the stronger the evidence of interaction, the more helpful it is to present stratum-specific estimates rather than an overall estimate of effect. However, there is no absolute cut-off, and it is up to us to decide when the interaction is so important to your main question of interest that you cannot present an overall estimate of effect. We cover these choices in more detail in Session SM12 – Strategies of Analysis. 
 

## Obtaining stratum–specific estimates using Stata

We have now seen how to calculate stratum-specific effects using the coefficients from 
the logistic regression model. Stata can produce stratum-specific results in the following 
ways: 
 
1. `R` can obtain stratum-specific estimates by producing a logistic model for each level of the stratifying variable. For example, to produce estimates for each level of agebin, type:

In [ ]:
anaemia_hookworm_agebin_stratum0_glm <-
glm(anaemic_sev ~ hookworm,
    data = helminths_df_3 %>% subset(agebin == 0),
    family = binomial)

In [ ]:
summary(anaemia_hookworm_agebin_stratum0_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_hookworm_agebin_stratum0_glm), 
    confint(anaemia_hookworm_agebin_stratum0_glm)))

In [ ]:
anaemia_hookworm_agebin_stratum1_glm <-
glm(anaemic_sev ~ hookworm,
    data = helminths_df_3 %>% subset(agebin == 1),
    family = binomial)

In [ ]:
summary(anaemia_hookworm_agebin_stratum1_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_hookworm_agebin_stratum1_glm), 
    confint(anaemia_hookworm_agebin_stratum1_glm)))

The disadvantage of this approach is that each model has a smaller sample size hence, less power, than a single model. The next two methods allow the full dataset to be used. 

2) The recommended approach when using the full dataset is to use linear combinations. There are readymade packages in `R` such as `lincom` from `biostat3` that compute point estimates, standard errors, t or z statistics,  p-values, and confidence intervals for linear combinations of coefficients of any regression model. We'll try and recreate these outputs using the function `glht` from the package `multcomp`.

Once again, let’s look at the output from our interaction model:

In [ ]:
summary(anaemia_agebin_interaction_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_agebin_interaction_glm), 
    confint(anaemia_agebin_interaction_glm)))

We have already calculated the odds ratio for hookworm infection vs no infection in the 25+ age group as OR = 2.95 × 0.88 = 2.60, i.e. by combining the main effects of hookworm and the interaction between agebin and hookworm. 

`R` can do this, but with the additional benefit of computing 95% confidence intervals: 

In [ ]:
library(multcomp)

In [ ]:
anaemia_agebin_interaction_lincom <-
    glht(anaemia_agebin_interaction_glm,
    linfct = c("hookworm + agebin1:hookworm = 1"))

In [ ]:
summary(anaemia_agebin_interaction_lincom)

In [ ]:
exp(cbind(OR = coef(anaemia_agebin_interaction_lincom),
    confint.default(anaemia_agebin_interaction_lincom)))

In this example, both `hookworm` and `agebin` are coded as binary variables. Suppose instead we modelled an interaction between `hookworm` and `agegrp`, where `agegrp` was coded 1, 2 or 3, then `R` would create the interaction terms `agebin2:hookworm` and `agebin3:hookworm`. Any of these could be combined using `glht` to compute stratum-specific odds ratios for the multiple strata. 

3) There is a third possible approach, which is to create new indicator variables using a method called ‘re-parameterisation’ and so obtain the stratum-specific estimates directly from the Stata output:


In [ ]:
helminths_df_4 <-
helminths_df_3 %>%
    mutate(hookwormage0 = if_else(agebin == 0 & hookworm == 1, 1, 0))

This will take the value 1 when hookworm = 1 and agebin = 0, otherwise it takes the value 0 (i.e. this corresponds to our stratum [C]). Similarly, 

In [ ]:
helminths_df_5 <-
helminths_df_4 %>%
    mutate(hookwormage1 = if_else(agebin == 1 & hookworm == 1, 1, 0))

This will take the value 1 when hookworm=1 & agebin=1, otherwise it takes the value 0 (i.e. this corresponds to our stratum [D]).

We can obtain age-group-specific effects for hookworm by fitting the model:

In [ ]:
anaemia_agebin_hookwormage_glm <-
glm(anaemic_sev ~ agebin + hookwormage0 + hookwormage1,
    data = helminths_df_5,
    family = binomial)

In [ ]:
summary(anaemia_agebin_hookwormage_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_agebin_hookwormage_glm), confint(anaemia_agebin_hookwormage_glm)))

To compare these results with the model including interaction you produced earlier, type:

In [ ]:
summary(anaemia_agebin_interaction_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_agebin_interaction_glm), confint(anaemia_agebin_interaction_glm)))

In [ ]:
logLik(anaemia_agebin_hookwormage_glm)
logLik(anaemia_agebin_interaction_glm)

Note that the models are identical, the log-likelihoods are equal (-818.24).  In using the new variable `hookwormage` we produced a model that directly gives the stratum-specific odds ratios for hookworm infection by each age-group <25 years and 25+ years, as well as the 95% confidence intervals for these odds ratios.  

Re-parameterising the model in this way can be useful if there is evidence of interaction (using a likelihood ratio test) and we want to obtain stratum-specific estimates. However it is probably more straightforward just to use the original interaction model followed by `glht` as described previously. 

## Presenting stratum–specific estimates

When there is evidence of interaction we may decide that it is more informative to present stratum-specific estimates for the effects of exposure on outcome, rather than an overall estimate of effect. 

However, as a result the interpretation and presentation of results from an interaction model is much less straightforward. We will now fit a single model that includes an interaction term for agebin and hookworm, plus two other variables (malaria and education), and then use Stata’s lincom command to compute stratum-specific odds 
ratios:

In [ ]:
anaemia_hookworm_agebin_malaria_edu_glm <-
glm(anaemic_sev ~ agebin*hookworm + malaria + as.factor(edu_cat),
    data = helminths_df_5,
    family = binomial)

In [ ]:
summary(anaemia_hookworm_agebin_malaria_edu_glm)

In [ ]:
exp(cbind(OR = coef(anaemia_hookworm_agebin_malaria_edu_glm), 
    confint(anaemia_hookworm_agebin_malaria_edu_glm)))

In [ ]:
anaemia_hookworm_agebin_malaria_edu_lincom <-
    glht(anaemia_hookworm_agebin_malaria_edu_glm,
    linfct = c("hookworm + agebin1:hookworm = 1"))

In [ ]:
summary(anaemia_hookworm_agebin_malaria_edu_lincom)

In [ ]:
exp(cbind(OR = coef(anaemia_hookworm_agebin_malaria_edu_lincom),
    confint.default(anaemia_hookworm_agebin_malaria_edu_lincom)))

In [ ]:
anaemia_hookworm_agebin_malaria_edu_lincom_2 <-
    glht(anaemia_hookworm_agebin_malaria_edu_glm,
    linfct = c("agebin1 + agebin1:hookworm = 1"))

In [ ]:
summary(anaemia_hookworm_agebin_malaria_edu_lincom_2)

In [ ]:
exp(cbind(OR = coef(anaemia_hookworm_agebin_malaria_edu_lincom_2),
    confint.default(anaemia_hookworm_agebin_malaria_edu_lincom_2)))

All of these logistic regression results can be presented in a single table as follows: 


|                                                        |                      | Adjusted odds ratio  (95% confidence interval) | P-value |
|--------------------------------------------------------|----------------------|------------------------------------------------|---------|
| Effect of hookworm in women  aged <25 years            | Hookworm  uninfected | 1.00                                           |         |
|                                                        | Hookworm  infected   | 2.89 (2.08, 4.02)                              | <0.001  |
| Effect of hookworm in women  aged 25+ years            | Hookworm  uninfected | 1.00                                           |         |
|                                                        | Hookworm  infected   | 2.33 (1.43, 3.78)                              | 0.001   |
| Effect of age in hookworm uninfected women             | <25 years            | 1.00                                           |         |
|                                                        | 25+ years            | 0.87 (0.56, 1.36)                              | 0.547   |
| Effect of age in hookworm infected women               | <25 years            | 1.00                                           |         |
|                                                        | 25+ years            | 0.70 (0.48, 1.04)                              | 0.076   |
| Effect on severe anaemia for other variables in model: |                      |                                                |         |
| Malaria                                                | Malaria  uninfected  | 1.00                                           |         |
|                                                        | Malaria infected     | 2.98 (2.16, 4.12)                              | <0.001  |
| Education                                              | None                 | 1.00                                           | 0.610*  |
|                                                        | Primary              | 1.01 (0.52, 1.97)                              |         |
|                                                        | Secondary            | 0.99 (0.50, 1.94)                              |         |
|                                                        | Tertiary             | 0.68 (0.29, 1.61)                              |         |

## Key points 

1) Interaction terms should be examined and tested using the likelihood ratio test. 
 
2) Stratum-specific estimates can be obtained by multiplying the relevant parameter estimates (`R`'s `ghlt` command can do this for you), by restricting the analysis to one stratum (category), or by creating new indicator variables which directly give stratum-specific estimates (‘re-parameterisation’).
 
3) To include an interaction term in a logistic model you should place a `*` between the two interacting variables.

## Review exercise 

Now try to carry out the same analyses on your own.  For this exercise you should use the mwanza dataset, a case-control study of HIV infection.  The solutions are given in Section 4.   
 
### 1) Using the mwanza data, investigate whether there is interaction between the effects of education and age. (You may need to combine categories for each variable.)

In [ ]:
mwanza_df <- read_dta("Data_files-20211113/MWANZA.dta")

In [ ]:
colnames(mwanza_df)

In [ ]:
mwanza_df_2 <- mwanza_df %>%
    mutate(ed = as.factor(ed)) %>%
    mutate(age1 = as.factor(age1))

In [ ]:
case_education_age1_interaction_glm <-
glm(case ~ ed * age1,
    data = mwanza_df_2,
    family = binomial)

In [ ]:
summary(case_education_age1_interaction_glm)

In [ ]:
exp(cbind(OR = coef(case_education_age1_interaction_glm), confint(case_education_age1_interaction_glm)))

#### Too many categories with some groups too small, so need to combine some 

In [ ]:
mwanza_df_3 <- mwanza_df_2 %>%
    mutate(ed = as.factor(case_when(ed ==1 ~ 1,
                                    ed !=1 ~ 2))) %>%
    mutate(age1 = as.factor(case_when(age1 == 1 ~ 1,
                                      age1 == 2 ~ 2,
                                      age1 == 3 ~ 2,
                                      age1 == 4 ~ 3,
                                      age1 == 5 ~ 3,
                                      age1 == 6 ~ 4)))

In [ ]:
case_education_age1_interaction_glm_2 <-
glm(case ~ ed * age1,
    data = mwanza_df_3,
    family = binomial)

In [ ]:
summary(case_education_age1_interaction_glm_2)

In [ ]:
exp(cbind(OR = coef(case_education_age1_interaction_glm_2), confint(case_education_age1_interaction_glm_2)))

In [ ]:
case_education_age1_glm <-
glm(case ~ ed + age1,
    data = mwanza_df_3,
    family = binomial)

In [ ]:
lrtest(case_education_age1_glm, case_education_age1_interaction_glm_2)

#### The model with the interaction term does not significantly improve the model

### 2) Assume there is interaction, and derive the appropriate age-specific odd ratios. 

In [ ]:
case_education_age1_lincom <-
    glht(case_education_age1_interaction_glm_2,
    linfct = c("ed2 + ed2:age12 = 1"))

In [ ]:
summary(case_education_age1_lincom)

In [ ]:
exp(cbind(OR = coef(case_education_age1_lincom),
    confint.default(case_education_age1_lincom)))

In [ ]:
case_education_age1_lincom2 <-
    glht(case_education_age1_interaction_glm_2,
    linfct = c("ed2 + ed2:age13 = 1"))

In [ ]:
summary(case_education_age1_lincom2)

In [ ]:
exp(cbind(OR = coef(case_education_age1_lincom2),
    confint.default(case_education_age1_lincom2)))

In [ ]:
case_education_age1_lincom3 <-
    glht(case_education_age1_interaction_glm_2,
    linfct = c("ed2 + ed2:age14 = 1"))

In [ ]:
summary(case_education_age1_lincom3)

In [ ]:
exp(cbind(OR = coef(case_education_age1_lincom3),
    confint.default(case_education_age1_lincom3)))

Adjusted OR for:

Age group 1, Education level 1 = 0.519

Age group 2,  Education level 1 = 1.952

Age group 3,  Education level 1 = 3.459

Age group 4,  Education level 1 = 2.847